# Лабораторная работа №7. "Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации"

*Задание №1.*
Решить задачи регрессии и классификации на данных в соответствии с Вашим индивидуальным вариантом (см. Лаб.работы №3, 4), используя полносвязные НС; реализовать НС посредством API Keras и фреймворка TensorFlow; оценить качество полученных моделей с помощью метрик.

*Задание №2.*
Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.


*Дополнительное Задание №3*.*
1. Самостоятельно изучить отличия работы оптимизаторов Adam и RMSProp от классического градиентного спуска.
2. Реализовать градиентный спуск с использованием указанных оптимизаторов; предусмотрите возможность использования реализованных вами оптими-заторов в Вашем персептроне.

In [27]:
import pandas as pd
import numpy as np
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

## Задание №1

### Загрузка и разбиение датасетов на выборки 

In [28]:
data_regression = pd.read_csv("filteredNotes\Lab3.csv")
data_classification = pd.read_csv("filteredNotes\Lab4.csv")

In [29]:
y_regression = data_regression["price"]
X_regression = data_regression.drop(["price", "Unnamed: 0"], axis=1)
y_classification = data_classification["bomb_planted"]
X_classification = data_classification.drop(["bomb_planted", "Unnamed: 0"], axis=1)

In [30]:
data_regression

,Unnamed: 0,price,bedrooms,bathrooms,sqft_living,floors,waterfront,view,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
0,0,221900.0,3,1.00,1180.0,1,0,0,7,1180,0,1955,0,1340,5650
1,1,538000.0,3,2.25,2570.0,2,0,0,7,2170,400,1951,1991,1690,7639
2,2,180000.0,2,1.00,770.0,1,0,0,6,770,0,1933,0,2720,8062
3,3,604000.0,4,3.00,1960.0,1,0,0,7,1050,910,1965,0,1360,5000
4,4,510000.0,3,2.00,1680.0,1,0,0,8,1680,0,1987,0,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21394,21608,360000.0,3,2.50,1530.0,3,0,0,8,1530,0,2009,0,1530,1509
21395,21609,400000.0,4,2.50,2310.0,2,0,0,8,2310,0,2014,0,1830,7200
21396,21610,402101.0,2,0.75,1020.0,2,0,0,7,1020,0,2009,0,1020,2007
21397,21611,400000.0,3,2.50,1600.0,2,0,0,8,1600,0,2004,0,1410,1287


In [31]:
(
    X_regression_train,
    X_regression_test,
    y_regression_train,
    y_regression_test,
) = train_test_split(X_regression, y_regression, test_size=0.2)
(
    X_classification_train,
    X_classification_test,
    y_classification_train,
    y_classification_test,
) = train_test_split(
    X_classification, y_classification, stratify=y_classification, test_size=0.2
)

In [32]:
X_classification_train

,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive
41121,61.87,15,14,300,200,298,200,10050,550,3,2,3,3,2
51015,114.93,1,0,500,500,499,500,2200,1050,4,3,1,5,5
84776,113.09,6,9,500,500,500,400,700,700,0,0,0,5,5
86984,99.20,9,15,500,500,490,500,1300,3900,2,5,2,5,5
59224,54.95,10,0,300,88,282,95,24100,300,3,1,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76072,32.16,12,8,200,230,200,279,1950,10800,2,3,1,2,3
49372,74.94,0,0,200,186,100,192,850,1200,0,0,1,2,2
90134,174.93,0,1,500,500,100,169,10650,19650,0,0,0,5,5
21689,31.66,0,1,332,206,375,346,400,1100,1,4,0,4,4


### Обучение модели регрессии

In [33]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(13,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [34]:
model_regression.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 64)                896       
                                                                 
 dense_35 (Dense)            (None, 32)                2080      
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                                 
 dense_36 (Dense)            (None, 16)                528       
                                                                 
 dropout_8 (Dropout)         (None, 16)                0         
                                                                 
 dense_37 (Dense)            (None, 1)                 17        
                                                                 
Total params: 3521 (13.75 KB)
Trainable params: 3521 (

In [35]:
model_regression.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse"
)

In [36]:
model_regression.fit(X_regression_train, y_regression_train, epochs=30)

Epoch 1/30


535/535 [==============================] - 2s 2ms/step - loss: 92970385408.0000
Epoch 2/30
535/535 [==============================] - 1s 2ms/step - loss: 64175902720.0000
Epoch 3/30
535/535 [==============================] - 1s 2ms/step - loss: 63264055296.0000
Epoch 4/30
535/535 [==============================] - 1s 2ms/step - loss: 60346753024.0000
Epoch 5/30
535/535 [==============================] - 1s 2ms/step - loss: 60343005184.0000
Epoch 6/30
535/535 [==============================] - 1s 2ms/step - loss: 58304458752.0000
Epoch 7/30
535/535 [==============================] - 2s 4ms/step - loss: 57851113472.0000
Epoch 8/30
535/535 [==============================] - 3s 5ms/step - loss: 57205301248.0000
Epoch 9/30
535/535 [==============================] - 2s 4ms/step - loss: 56999919616.0000
Epoch 10/30
535/535 [==============================] - 3s 5ms/step - loss: 55080259584.0000
Epoch 11/30
535/535 [==============================] - 2s 4ms/step - loss: 55102373888.0000
Epoch 12

In [37]:
print(
    mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test))
)
print(
    mean_squared_error(y_regression_test, model_regression.predict(X_regression_test))
)

134/134 [==============================] - 1s 3ms/step
140378.6386463201
134/134 [==============================] - 0s 3ms/step
45761201947.00683


отработало плохо

### Обучение модели классификации

In [38]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse"
)
# verbose=None - не будет логов
model_classification_1.fit(
    X_classification_train, y_classification_train, epochs=25, verbose=None
)

In [39]:
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94     19752
           1       0.00      0.00      0.00      2646

    accuracy                           0.88     22398
   macro avg       0.44      0.50      0.47     22398
weighted avg       0.78      0.88      0.83     22398

[[19752     0]
 [ 2646     0]]


c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

результаты не очень - очень много ошибок второго рода

In [40]:
w0 = 1 / y_classification_train[y_classification_train == 0].shape[0]
w1 = 1 / y_classification_train[y_classification_train == 1].shape[0]

In [41]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model_classification_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy"
)
model_classification_1.fit(
    X_classification_train,
    y_classification_train,
    epochs=25,
    verbose=None,
    class_weight={0: w0, 1: w1},
)
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94     19752
           1       0.54      0.99      0.70      2646

    accuracy                           0.90     22398
   macro avg       0.77      0.94      0.82     22398
weighted avg       0.94      0.90      0.91     22398

[[17524  2228]
 [   23  2623]]


Стало гораздно лучше

In [42]:
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 2 нейрона и softmax
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)
# в качестве функции активации используется категориальная кроссэнтропия
# используем разряженный (sparse) вариант, поскольку значения целевого признака не закодированы One-Hot кодированием
model_classification_2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
    loss="sparse_categorical_crossentropy",
)
model_classification_2.fit(
    X_classification_train,
    y_classification_train,
    epochs=25,
    verbose=None,
    class_weight={0: w0, 1: w1},
)

In [43]:
model_classification_2.predict(X_classification_test, verbose=None)[:5]

array([[0.52042484, 0.47957516],
       [0.52042484, 0.47957516],
       [0.52042484, 0.47957516],
       [0.52042484, 0.47957516],
       [0.52042484, 0.47957516]], dtype=float32)

In [44]:
y_pred = [
    np.argmax(pred)
    for pred in model_classification_2.predict(X_classification_test, verbose=None)
]

In [45]:
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94     19752
           1       0.00      0.00      0.00      2646

    accuracy                           0.88     22398
   macro avg       0.44      0.50      0.47     22398
weighted avg       0.78      0.88      0.83     22398

[[19752     0]
 [ 2646     0]]


c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [46]:
import pickle
    
model_classification_1.save('C:/MLforGIt/ML_RGR/Models/NN.h5')

c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Тут напротив - большой завал в один из классов

Попробуем устранить дисбаланс с помощью NearMiss

In [47]:
print(f"1: {sum(y_classification_train == 1)}")
print(f"0: {sum(y_classification_train == 0)}")

1: 10582
0: 79008


In [48]:
nm = NearMiss()
X_classification_train, y_classification_train = nm.fit_resample(
    X_classification_train, y_classification_train.ravel()
)

In [49]:
print(f"1: {sum(y_classification_train == 1)}")
print(f"0: {sum(y_classification_train == 0)}")

1: 10582
0: 10582


In [50]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse"
)
# verbose=None - не будет логов
model_classification_1.fit(
    X_classification_train, y_classification_train, epochs=25, verbose=None
)

In [51]:
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     19752
           1       0.12      1.00      0.21      2646

    accuracy                           0.12     22398
   macro avg       0.06      0.50      0.11     22398
weighted avg       0.01      0.12      0.02     22398

[[    0 19752]
 [    0  2646]]


c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

с binary_crossentropy получилось лучше всего

In [52]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model_classification_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy"
)
model_classification_1.fit(
    X_classification_train,
    y_classification_train,
    epochs=25,
    verbose=None,
    class_weight={0: w0, 1: w1},
)
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.81      0.89     19752
           1       0.41      1.00      0.58      2646

    accuracy                           0.83     22398
   macro avg       0.70      0.90      0.74     22398
weighted avg       0.93      0.83      0.86     22398

[[15937  3815]
 [    2  2644]]
